# Ch22: 除錯技術 | Debugging Techniques

## 📚 課程大綱

**學習時數**：80 分鐘（理論 + 6 個範例）

**本課程包含**：
1. 除錯的重要性與基本概念
2. 範例 1：Print Debugging 技巧
3. 範例 2：logging 模組基礎
4. 範例 3：logging 進階應用（檔案輸出、格式化）
5. 範例 4：pdb 除錯器入門
6. 範例 5：breakpoint() 與 pdb 進階指令
7. 範例 6：科學除錯法四步驟實戰

---

## 第一部分：理論基礎（20 分鐘）

### 1.1 為什麼需要除錯？

**現實情境**：
- 程式寫完後發現結果不對，但不知道哪裡錯了
- 程式執行時突然崩潰，看不懂錯誤訊息
- 修改一個 bug 後又出現新的 bug

**系統化除錯的價值**：
1. **提高效率**：10 分鐘找到 bug vs 3 小時亂試
2. **可重現性**：能記錄並分享除錯過程
3. **科學方法**：基於證據而非猜測
4. **預防未來錯誤**：了解錯誤模式

### 1.2 錯誤的三種類型

| 錯誤類型 | 英文 | 發現時機 | 特徵 | 範例 |
|:---------|:-----|:---------|:-----|:-----|
| 語法錯誤 | Syntax Error | 執行前 | Python 直接報錯 | `if x = 10:` |
| 執行時期錯誤 | Runtime Error | 執行時 | 拋出例外 | `1 / 0` |
| 邏輯錯誤 | Logic Error | 使用時 | 結果錯誤但不崩潰 | 計算平均值忘記除以數量 |

**最難除錯的是邏輯錯誤**：因為程式不會報錯，只是結果不對！

### 1.3 除錯方法的演進

```
Level 1: Print Debugging
  優點：簡單、直覺
  缺點：侵入性、難管理、要手動刪除

Level 2: Logging
  優點：分級控制、可保存、可關閉
  缺點：需要學習新模組

Level 3: Debugger (pdb)
  優點：互動式、全功能、可暫停執行
  缺點：需要熟悉指令
```

**建議**：
- 快速測試 → print debugging
- 長期維護 → logging
- 複雜邏輯 → pdb

---

## 第二部分：實作演練（55 分鐘）

### 範例 1：Print Debugging 技巧（8 分鐘）

**學習目標**：掌握有效的 print 除錯技巧

**問題情境**：計算列表中所有偶數的平方和

In [ ]:
# 範例 1-1：基本 Print Debugging

# ❌ 有 bug 的版本
def sum_of_even_squares_buggy(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            square = num * 2  # Bug: 應該是 num ** 2
            total += square
    return total

# 測試
numbers = [1, 2, 3, 4, 5]
result = sum_of_even_squares_buggy(numbers)
print(f"結果: {result}")  # 期望: 20 (2²+4²=4+16=20)，實際: 12

In [ ]:
# 範例 1-2：使用 print 追蹤變數

def sum_of_even_squares_debug(numbers):
    print(f"[DEBUG] 輸入: {numbers}")  # 追蹤輸入
    total = 0
    for num in numbers:
        print(f"[DEBUG] 處理數字: {num}")  # 追蹤迴圈
        if num % 2 == 0:
            print(f"[DEBUG]   {num} 是偶數")  # 追蹤條件
            square = num * 2  # Bug 仍在
            print(f"[DEBUG]   平方: {square}")  # 發現問題！
            total += square
    print(f"[DEBUG] 最終結果: {total}")
    return total

result = sum_of_even_squares_debug(numbers)
# 透過輸出可以看到：2 的平方應該是 4，但這裡是 4（2*2 而非 2**2）

In [ ]:
# 範例 1-3：✅ 修正後的版本

def sum_of_even_squares(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            square = num ** 2  # 修正：使用 **2 計算平方
            total += square
    return total

result = sum_of_even_squares(numbers)
print(f"修正後結果: {result}")  # 20 ✓

# Python 3.8+ 技巧：使用 f-string = 語法
x = 10
print(f"{x=}")  # 輸出：x=10（同時顯示變數名和值）

**範例 1 總結**：
- Print debugging 適合快速測試
- 使用 `[DEBUG]` 前綴方便識別除錯訊息
- 使用縮排顯示程式邏輯層次
- **缺點**：程式碼會變得很亂，發布前要全部刪除

---

### 範例 2：logging 模組基礎（10 分鐘）

**學習目標**：使用 logging 模組取代 print

**logging 的五個級別**：
- **DEBUG** (10)：詳細診斷資訊（開發時用）
- **INFO** (20)：一般資訊（確認程式運作正常）
- **WARNING** (30)：警告訊息（可能出問題）
- **ERROR** (40)：錯誤訊息（程式某部分失敗）
- **CRITICAL** (50)：嚴重錯誤（程式可能崩潰）

In [ ]:
# 範例 2-1：logging 基本使用

import logging

# 設定 logging 級別
logging.basicConfig(level=logging.DEBUG)  # 顯示 DEBUG 及以上級別

# 測試所有級別
logging.debug("這是 DEBUG 訊息 - 詳細的除錯資訊")
logging.info("這是 INFO 訊息 - 一般執行資訊")
logging.warning("這是 WARNING 訊息 - 警告訊息")
logging.error("這是 ERROR 訊息 - 錯誤發生")
logging.critical("這是 CRITICAL 訊息 - 嚴重錯誤")

In [ ]:
# 範例 2-2：使用 logging 改寫範例 1

import logging

# 重新設定 logging（清除之前的設定）
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)s - %(message)s'  # 自訂格式
)

def sum_of_even_squares_logging(numbers):
    logging.debug(f"輸入: {numbers}")
    total = 0
    for num in numbers:
        logging.debug(f"處理數字: {num}")
        if num % 2 == 0:
            logging.info(f"{num} 是偶數")
            square = num ** 2
            logging.debug(f"平方: {square}")
            total += square
    logging.info(f"最終結果: {total}")
    return total

result = sum_of_even_squares_logging([1, 2, 3, 4, 5])

In [ ]:
# 範例 2-3：調整級別（發布時使用）

# 只顯示 WARNING 以上的訊息
logging.basicConfig(
    level=logging.WARNING,
    format='%(levelname)s - %(message)s',
    force=True  # 強制重新設定
)

# 再次執行，DEBUG 和 INFO 都不會顯示
result = sum_of_even_squares_logging([1, 2, 3, 4, 5])
print(f"\n結果: {result}")  # 只看到結果，沒有除錯訊息

**範例 2 總結**：
- logging 不會混入正常輸出
- 可以透過級別控制顯示的訊息
- 發布時只需改變級別，不用刪除程式碼
- **級別選擇建議**：
  - 開發時：`DEBUG` 或 `INFO`
  - 測試時：`INFO` 或 `WARNING`
  - 生產環境：`WARNING` 或 `ERROR`

---

### 範例 3：logging 進階應用（10 分鐘）

**學習目標**：將 log 寫入檔案、自訂格式

In [ ]:
# 範例 3-1：將 log 寫入檔案

import logging

# 設定同時輸出到螢幕和檔案
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('debug.log', mode='w', encoding='utf-8'),
        logging.StreamHandler()  # 同時顯示在螢幕
    ],
    force=True
)

logging.debug("這是除錯訊息")
logging.info("這是資訊訊息")
logging.warning("這是警告訊息")

print("\n檢查 debug.log 檔案，所有訊息都被記錄了！")

In [ ]:
# 範例 3-2：讀取 log 檔案

with open('debug.log', 'r', encoding='utf-8') as f:
    print("=== debug.log 內容 ===")
    print(f.read())

In [ ]:
# 範例 3-3：實際應用案例 - 計算器程式

import logging

# 建立 logger
logger = logging.getLogger('calculator')
logger.setLevel(logging.DEBUG)

# 建立 handler
file_handler = logging.FileHandler('calculator.log', mode='w', encoding='utf-8')
console_handler = logging.StreamHandler()

# 設定格式
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# 加入 handler
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# 計算器函式
def divide(a, b):
    logger.info(f"計算 {a} / {b}")
    try:
        result = a / b
        logger.debug(f"結果: {result}")
        return result
    except ZeroDivisionError:
        logger.error("除以零錯誤！")
        return None
    except Exception as e:
        logger.critical(f"未預期的錯誤: {e}")
        return None

# 測試
print(divide(10, 2))
print(divide(10, 0))  # 會產生錯誤 log
print(divide("10", 2))  # 會產生嚴重錯誤 log

**範例 3 總結**：
- 使用 `FileHandler` 將 log 寫入檔案
- 可以同時輸出到多個目標（檔案 + 螢幕）
- 自訂格式可以包含：時間、級別、訊息、函式名等
- **最佳實踐**：
  - 使用 `logger = logging.getLogger(__name__)` 建立 logger
  - 不同模組使用不同名稱的 logger
  - 生產環境將 log 寫入檔案，方便事後分析

---

### 範例 4：pdb 除錯器入門（10 分鐘）

**學習目標**：使用 pdb 互動式除錯

**pdb 是什麼**：Python Debugger，可以暫停程式執行、檢視變數、單步執行

**啟動 pdb 的方法**：
1. `import pdb; pdb.set_trace()` (Python 3.6 以前)
2. `breakpoint()` (Python 3.7+ 推薦)
3. `python -m pdb script.py` (從命令列)

In [ ]:
# 範例 4-1：基本 pdb 使用（請在命令列執行）

def calculate_factorial(n):
    """計算階乘"""
    if n < 0:
        return None
    if n == 0:
        return 1
    
    result = 1
    for i in range(1, n + 1):
        # breakpoint()  # 取消註解後執行，程式會在這裡暫停
        result *= i
    return result

# 測試
print(calculate_factorial(5))  # 應該是 120

**pdb 常用指令**（請實際操作時記住）：

| 指令 | 全名 | 功能 | 範例 |
|:-----|:-----|:-----|:-----|
| `l` | list | 顯示當前程式碼 | `(Pdb) l` |
| `p` | print | 印出變數值 | `(Pdb) p result` |
| `pp` | pprint | 美化印出 | `(Pdb) pp data` |
| `n` | next | 執行下一行 | `(Pdb) n` |
| `s` | step | 進入函式 | `(Pdb) s` |
| `c` | continue | 繼續執行 | `(Pdb) c` |
| `w` | where | 顯示呼叫堆疊 | `(Pdb) w` |
| `u` | up | 移到上層 | `(Pdb) u` |
| `d` | down | 移到下層 | `(Pdb) d` |
| `q` | quit | 離開除錯器 | `(Pdb) q` |

**記憶口訣**：「**p**rint 查看，**n**ext 前進，**s**tep 深入，**c**ontinue 跳過」

In [ ]:
# 範例 4-2：使用 pdb 找出 bug

def find_max_buggy(numbers):
    """找出列表中的最大值（有 bug）"""
    max_val = 0  # Bug: 如果所有數字都是負數會錯誤
    for num in numbers:
        # breakpoint()  # 取消註解，觀察 max_val 的變化
        if num > max_val:
            max_val = num
    return max_val

# 測試
print(find_max_buggy([3, 5, 2, 8, 1]))  # 正確：8
print(find_max_buggy([-3, -5, -2, -8, -1]))  # 錯誤：0（應該是 -1）

In [ ]:
# 範例 4-3：✅ 修正後的版本

def find_max(numbers):
    """找出列表中的最大值（修正後）"""
    if not numbers:
        return None
    
    max_val = numbers[0]  # 修正：使用第一個元素初始化
    for num in numbers:
        if num > max_val:
            max_val = num
    return max_val

# 測試
print(find_max([3, 5, 2, 8, 1]))  # 8
print(find_max([-3, -5, -2, -8, -1]))  # -1 ✓

**範例 4 總結**：
- pdb 可以暫停程式執行，逐步觀察變數變化
- 使用 `breakpoint()` 設定中斷點（Python 3.7+）
- 在 Jupyter Notebook 中使用 pdb 可能會有限制，建議在命令列執行
- **使用時機**：邏輯複雜、變數多、print 不夠用時

---

### 範例 5：breakpoint() 與 pdb 進階指令（10 分鐘）

**學習目標**：掌握 pdb 的進階用法

In [ ]:
# 範例 5-1：條件中斷點

def sum_range(n):
    """計算 1 到 n 的總和"""
    total = 0
    for i in range(1, n + 1):
        # 只在 i == 5 時中斷
        if i == 5:
            # breakpoint()  # 取消註解測試
            pass
        total += i
    return total

print(sum_range(10))  # 應該是 55

In [ ]:
# 範例 5-2：檢視呼叫堆疊（多層函式呼叫）

def level_3():
    x = 30
    # breakpoint()  # 在這裡中斷，可以用 w 查看呼叫堆疊
    return x * 3

def level_2():
    y = 20
    return level_3() + y

def level_1():
    z = 10
    return level_2() + z

result = level_1()
print(f"結果: {result}")

# 在 breakpoint() 處使用的指令：
# (Pdb) w      # 查看呼叫堆疊：level_1 → level_2 → level_3
# (Pdb) u      # 移到上一層（level_2）
# (Pdb) p y    # 印出 level_2 的變數 y
# (Pdb) u      # 再移到上一層（level_1）
# (Pdb) p z    # 印出 level_1 的變數 z
# (Pdb) d      # 回到 level_3
# (Pdb) c      # 繼續執行

In [ ]:
# 範例 5-3：在 pdb 中執行任意 Python 程式碼

def process_list(numbers):
    result = []
    for num in numbers:
        # breakpoint()  # 在這裡可以測試不同的處理方式
        # 在 pdb 中可以輸入：
        # (Pdb) result.append(num * 2)  # 測試加倍
        # (Pdb) result.append(num ** 2)  # 測試平方
        # (Pdb) p result  # 查看當前結果
        result.append(num * 2)
    return result

print(process_list([1, 2, 3, 4, 5]))

**範例 5 總結**：
- 使用條件判斷只在特定情況下中斷（避免遞迴時中斷太多次）
- `w` 指令查看呼叫堆疊，了解函式呼叫鏈
- `u` 和 `d` 在堆疊之間移動，檢視不同層級的變數
- 在 pdb 中可以執行任意 Python 程式碼，測試修正方案
- **最佳實踐**：
  - 使用 `!` 前綴強制執行變數名與指令衝突時（如 `!n` 印出變數 n）
  - 使用 `pp` 而非 `p` 印出複雜資料結構（字典、列表）

---

### 範例 6：科學除錯法四步驟實戰（12 分鐘）

**學習目標**：應用系統化除錯流程

**科學除錯法**：
1. **重現 (Reproduce)**：建立最小測試案例
2. **隔離 (Isolate)**：縮小問題範圍
3. **假設 (Hypothesize)**：提出可能原因
4. **驗證 (Validate)**：測試假設並修正

In [ ]:
# 範例 6：實際案例 - 學生成績統計系統

# ❌ 有 bug 的版本
class GradeCalculator:
    def __init__(self):
        self.grades = []
    
    def add_grade(self, grade):
        if 0 <= grade <= 100:
            self.grades.append(grade)
    
    def get_average(self):
        total = sum(self.grades)
        count = len(self.grades)
        return total / count  # Bug: 如果 grades 為空會出錯
    
    def get_letter_grade(self, score):
        if score >= 90:
            return 'A'
        elif score >= 80:
            return 'B'
        elif score >= 70:
            return 'C'
        elif score >= 60:
            return 'D'
        else:
            return 'F'
    
    def get_statistics(self):
        avg = self.get_average()
        letter = self.get_letter_grade(avg)
        return {
            'average': avg,
            'letter_grade': letter,
            'total_students': len(self.grades)
        }

# 測試（會出錯）
calc = GradeCalculator()
try:
    stats = calc.get_statistics()  # 會拋出 ZeroDivisionError
    print(stats)
except ZeroDivisionError as e:
    print(f"錯誤發生：{e}")

In [ ]:
# 步驟 1：重現 (Reproduce) - 建立最小測試案例

def minimal_reproduce():
    """最小重現案例"""
    grades = []  # 空列表
    total = sum(grades)  # 0
    count = len(grades)  # 0
    avg = total / count  # ZeroDivisionError!
    return avg

try:
    minimal_reproduce()
except ZeroDivisionError:
    print("✓ 成功重現錯誤：空列表時會除以零")

In [ ]:
# 步驟 2：隔離 (Isolate) - 確認問題出在哪一行

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s', force=True)

def get_average_debug(grades):
    logging.debug(f"[1] grades = {grades}")
    total = sum(grades)
    logging.debug(f"[2] total = {total}")
    count = len(grades)
    logging.debug(f"[3] count = {count}")  # count = 0
    logging.debug(f"[4] 即將執行：total / count = {total} / {count}")
    avg = total / count  # ← 問題出在這裡！
    logging.debug(f"[5] avg = {avg}")
    return avg

try:
    get_average_debug([])
except ZeroDivisionError:
    logging.error("✓ 隔離成功：問題出在第 4 行，count = 0 時除法會出錯")

In [ ]:
# 步驟 3：假設 (Hypothesize) - 提出可能的解決方案

print("可能的解決方案：")
print("假設 1：檢查 grades 是否為空，空的話返回 0")
print("假設 2：檢查 grades 是否為空，空的話返回 None")
print("假設 3：檢查 grades 是否為空，空的話拋出自訂例外")
print("\n選擇假設 2（返回 None 表示無法計算）")

In [ ]:
# 步驟 4：驗證 (Validate) - 測試修正方案

# ✅ 修正後的版本
class GradeCalculatorFixed:
    def __init__(self):
        self.grades = []
    
    def add_grade(self, grade):
        if 0 <= grade <= 100:
            self.grades.append(grade)
        else:
            logging.warning(f"無效的成績：{grade}（應在 0-100 之間）")
    
    def get_average(self):
        if not self.grades:  # 修正：檢查是否為空
            logging.warning("無法計算平均值：尚未輸入任何成績")
            return None
        total = sum(self.grades)
        count = len(self.grades)
        return total / count
    
    def get_letter_grade(self, score):
        if score is None:  # 修正：處理 None 的情況
            return 'N/A'
        if score >= 90:
            return 'A'
        elif score >= 80:
            return 'B'
        elif score >= 70:
            return 'C'
        elif score >= 60:
            return 'D'
        else:
            return 'F'
    
    def get_statistics(self):
        avg = self.get_average()
        letter = self.get_letter_grade(avg)
        return {
            'average': avg,
            'letter_grade': letter,
            'total_students': len(self.grades)
        }

# 測試案例
print("=== 測試 1：空列表 ===")
calc1 = GradeCalculatorFixed()
print(calc1.get_statistics())

print("\n=== 測試 2：正常資料 ===")
calc2 = GradeCalculatorFixed()
calc2.add_grade(85)
calc2.add_grade(92)
calc2.add_grade(78)
print(calc2.get_statistics())

print("\n=== 測試 3：無效資料 ===")
calc3 = GradeCalculatorFixed()
calc3.add_grade(120)  # 無效
calc3.add_grade(-10)  # 無效
calc3.add_grade(95)   # 有效
print(calc3.get_statistics())

print("\n✓ 驗證成功：所有測試案例都通過了！")

**範例 6 總結**：

**科學除錯法四步驟回顧**：
1. **重現**：建立最小測試案例 `grades = []` → 除以零
2. **隔離**：使用 logging 追蹤，發現問題在 `total / count`
3. **假設**：提出解決方案「檢查是否為空，返回 None」
4. **驗證**：執行多組測試案例，確認修正有效

**關鍵要點**：
- 不要猜測，要有證據支持（使用 logging/print/pdb 取得證據）
- 一次只修改一個假設（如果同時改多處，無法確認是哪個修正有效）
- 寫測試案例驗證修正（包含正常案例、邊界案例、錯誤案例）

---

## 第三部分：本章總結（5 分鐘）

### 知識回顧

**除錯方法比較**：

| 方法 | 優點 | 缺點 | 使用時機 |
|:-----|:-----|:-----|:---------|
| Print Debugging | 簡單、快速 | 侵入性、難管理 | 快速測試、簡單邏輯 |
| logging | 分級、可保存、可關閉 | 需學習 | 長期維護、生產環境 |
| pdb | 互動式、全功能 | 學習曲線 | 複雜邏輯、多層呼叫 |

**logging 級別**：
```
DEBUG    → 詳細診斷資訊（開發時）
INFO     → 一般資訊（確認運作正常）
WARNING  → 警告訊息（可能有問題）
ERROR    → 錯誤訊息（某部分失敗）
CRITICAL → 嚴重錯誤（程式可能崩潰）
```

**pdb 指令速查**：
- `p 變數名` - 查看變數
- `n` - 下一行（不進入函式）
- `s` - 進入函式
- `c` - 繼續執行
- `l` - 顯示程式碼
- `w` - 顯示呼叫堆疊
- `q` - 離開

**科學除錯法**：
1. 重現 - 建立最小測試案例
2. 隔離 - 縮小問題範圍
3. 假設 - 提出可能原因
4. 驗證 - 測試假設並修正

### 常見誤區

1. **過度依賴 print**：到處加 print，最後忘記刪除
   - 解決：改用 logging，可以控制顯示級別

2. **不會用 pdb**：進入 pdb 後不知道該做什麼
   - 解決：記住口訣「p 查看，n 前進，s 深入，c 跳過」

3. **亂猜測修改**：看到錯誤就隨便改，越改越糟
   - 解決：使用科學除錯法，有證據才修改

4. **logging 訊息不顯示**：忘記設定 level=DEBUG
   - 解決：`logging.basicConfig(level=logging.DEBUG)`

### 自我檢核

完成本章後，您應該能夠：
- [ ] 使用 print() 追蹤變數值
- [ ] 設定 logging 的基本配置
- [ ] 使用 logging 的五個級別
- [ ] 將 log 寫入檔案
- [ ] 使用 breakpoint() 設定中斷點
- [ ] 在 pdb 中使用 p, n, s, c, l, w 指令
- [ ] 應用科學除錯法四步驟
- [ ] 根據錯誤類型選擇適當的除錯方法

### 延伸閱讀

**官方文件**：
- [pdb — The Python Debugger](https://docs.python.org/3/library/pdb.html)
- [logging — Logging facility for Python](https://docs.python.org/3/library/logging.html)

**推薦資源**：
- [Real Python - Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/)
- [Python Logging HOWTO](https://docs.python.org/3/howto/logging.html)

**下一步**：
- 練習 `02-worked-examples.ipynb` 的 4 個完整案例
- 完成 `03-practice.ipynb` 的 8 題課堂練習
- 挑戰 `04-exercises.ipynb` 的 12 題習題

---

**學習提醒**：除錯是程式設計師最重要的技能！請確實練習每一個除錯工具。